In [ ]:
from hashlib import new
from requests import head
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import csv
import requests

# Song List Scraper

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.azlyrics.com/e/eminem.html")

songs = driver.find_elements(By.XPATH, "//div[@class='listalbum-item']/a")
song_list = []
for s in range(len(songs)):
    song_list.append(songs[s].text)

with open('song_list.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for s in range(len(song_list)):
        writer.writerow([song_list[s]])

# Get Lyrics

From lyrics.ovh API library by exhaustive GET requests

To an array of words (with duplicates -> frequency)

To a .csv of individual words.

And another transformation to pandas table

In [ ]:
bow = set()
lyrics_res = ""

with open('song_list_encoded.csv', 'r', newline='') as f:
    reader = csv.reader(f)

    song_list = set()
    # print reader
    for row in reader:
        song_list.add(row[0])

    for song in song_list:
        print("Retrieving: " + song)
        lyrics_res = requests.get(
            'https://api.lyrics.ovh/v1/Eminem/{song}').json()
        print(lyrics_res)
        try:
            bow.add(lyrics_res['lyrics'].split())
        except:
            print(f"Lyrics not found for {song}")
            continue

with open('bow.csv', 'w') as f:
    for word in bow:
        f.write(word + "\n")
    f.close()

# Cleaning Data

In [1]:
!sed -e "s/('s)//g" -e "s/[(''),]//g" data/bow.csv > data/bow_cleaned.csv

In [4]:
!sed -e 's/"//g' data/bow_cleaned.csv > data/bow_cleaned_2.csv

In [16]:
!sed -e 's/[?]//g' data/bow_cleaned.csv > data/bow_cleaned_3.csv

# Build dataframe from cleaned data

In [17]:
import pandas as pd
import numpy as np

# import bow_cleaned as dataframe
df = pd.read_csv('data/bow_cleaned.csv', header=None)

In [19]:
word_freq = df[0].value_counts()


In [39]:
word_freq

I              8814
the            8060
you            6216
a              5808
to             5697
               ... 
mufuckin          1
Bloodsuckin       1
suckubusses       1
bouts             1
satisfied         1
Name: 0, Length: 19437, dtype: int64

In [40]:
word_freq.to_csv('data/word_freq.csv')

In [41]:
word_freq = pd.read_csv('data/word_freq.csv', header=None)

In [42]:
word_freq

,0,1
0,I,8814
1,the,8060
2,you,6216
3,a,5808
4,to,5697
...,...,...
19432,mufuckin,1
19433,Bloodsuckin,1
19434,suckubusses,1
19435,bouts,1
